# 【問題1】公式チュートリアルモデルを分担して実行
TensorFLowの公式チュートリアルモデルを分担して実行してください。


以下の中から1人ひとつ選び実行し、その結果を簡単に発表してください。


models/tutorials at master · tensorflow/models

In [1]:
import io
import itertools
import numpy as np
import os
import re
import string
import tensorflow as tf
import tqdm

from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Dot, Embedding, Flatten, GlobalAveragePooling1D, Reshape
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [2]:
'''
season = ['Spring', 'Summer', 'Fall', 'Winter']
iter_season = iter(season)

print(type(iter_season)) # 型を表示して確認

print(next(iter_season)) # 1番目のイテレータを表示後、次のイテレータに進む
next(iter_season) # 次のイテレータに進む

# イテレータを１つずつ取り出して表示
for i in iter_season:
    print(i)
'''

"\nseason = ['Spring', 'Summer', 'Fall', 'Winter']\niter_season = iter(season)\n\nprint(type(iter_season)) # 型を表示して確認\n\nprint(next(iter_season)) # 1番目のイテレータを表示後、次のイテレータに進む\nnext(iter_season) # 次のイテレータに進む\n\n# イテレータを１つずつ取り出して表示\nfor i in iter_season:\n    print(i)\n"

In [3]:
SEED = 42 
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())
print(len(tokens))

8


In [5]:

# トークンから整数インデックスへのマッピングを保存するための語彙を作成

vocab, index = {}, 1 # start indexing from 1
vocab['<pad>'] = 0 # add a padding token 

for token in tokens:
    if token not in vocab: 
        vocab[token] = index
        index += 1
vocab_size = len(vocab)
print(vocab)
print(vocab_size)

{'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}
8


In [6]:
# dictの順番を入れ替える
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab) 

{0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


In [7]:
# 文章をベクトル化
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[1, 2, 3, 4, 5, 1, 6, 7]


In [8]:
# スキップグラムを生成する
# Word2Vecのデータ準備を簡素化する便利な関数を提供します。
# スキップぐらむは全てのベクトルの組み合わせを出力している

window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      example_sequence, 
      vocabulary_size=vocab_size, # 文章のトータルの長さ
      window_size=window_size, # 前後のコンテキストの数
      negative_samples=0, # ネガティブサンプリングの数
    shuffle=True) # ドロップアウト
print(len(positive_skip_grams))

positive_skip_grams

26


[[1, 4],
 [1, 7],
 [6, 1],
 [5, 1],
 [1, 6],
 [1, 5],
 [5, 4],
 [4, 5],
 [4, 1],
 [1, 3],
 [6, 5],
 [3, 2],
 [4, 3],
 [2, 1],
 [3, 1],
 [2, 4],
 [5, 6],
 [3, 4],
 [5, 3],
 [3, 5],
 [7, 1],
 [7, 6],
 [2, 3],
 [4, 2],
 [1, 2],
 [6, 7]]

In [9]:
# 例(さっきの奴を表示している)
for target, context in positive_skip_grams[:5]:
    print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(1, 4): (the, shimmered)
(1, 7): (the, sun)
(6, 1): (hot, the)
(5, 1): (in, the)
(1, 6): (the, hot)


In [10]:
# スキップぐらむからtarget、contex-wordを抽出
target_word, context_word = positive_skip_grams[0]

# ネガティブサンプリングの数
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class, # ポジティブクラス全体
    num_true=1, # ポジティブクラスの数？
    num_sampled=num_ns, # ネガティブサンプリングの数
    unique=True, # all the negative samples should be unique
    range_max=vocab_size, # 持ってくる特徴量の範囲
    seed=SEED, 
    name="negative_sampling" 
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

#tf.constant(context_word, dtype="int64")
#tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
context_class

tf.Tensor([2 1 4 3], shape=(4,), dtype=int64)
['wide', 'the', 'shimmered', 'road']


<tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[4]])>

In [11]:

# トレーニング例
# 次元を増やす
negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1)

# ポジティブワードとネガティブワードを合体させる
context = tf.concat([context_class, negative_sampling_candidates], 0)

# ラベルの作成
label = tf.constant([1] + [0]*num_ns, dtype="int64") 

# 次元を潰す
target = tf.squeeze(target_word)
context = tf.squeeze(context)
label =  tf.squeeze(label)


In [12]:
# labelの確認
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 1
target_word     : the
context_indices : [4 2 1 4 3]
context_words   : ['shimmered', 'wide', 'the', 'shimmered', 'road']
label           : [1 0 0 0 0]


In [13]:
# スキップグラムネガティブサンプリングWord2Vecモデルをトレーニングする
print(f"target  :", target)
print(f"context :", context )
print(f"label   :", label )

target  : tf.Tensor(1, shape=(), dtype=int32)
context : tf.Tensor([4 2 1 4 3], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


In [14]:
# 単語頻度ランクベースの確率的サンプリングテーブルを生成
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [15]:
# トレーニングデータを生成する

# スキップグラムとネガティブサンプリングのペアをシーケンスに与える
# ウィンドウズサイズは負のサンプルに基づく
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
    targets, contexts, labels = [], [], []
    # vocab_sizeトークンのサンプリングテーブルを作成
    sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)
    
    # データセット内のすべてのシーケンス(文)に反復
    for sequence in tqdm.tqdm(sequences):

    # シーケンス(文)の正のスキップグラムペアを生成します。.
        positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
            sequence, 
            vocabulary_size=vocab_size,
            sampling_table=sampling_table,
            window_size=window_size,
            negative_samples=0)

    # トレーニング例を作成するために、各ポジティブスキップグラムペアを反復します。 
    # 正のコンテキストワードと負のサンプル。.
        for target_word, context_word in positive_skip_grams:
            context_class = tf.expand_dims(tf.constant([context_word], dtype="int64"), 1)
            negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
                true_classes=context_class,
                num_true=1,
                num_sampled=num_ns, 
                unique=True, 
                range_max=vocab_size, 
                seed=SEED, 
                name="negative_sampling")

      # コンテキストを作成し、ベクトルにラベルを付けます(1つのターゲットワード)
            negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1)

            context = tf.concat([context_class, negative_sampling_candidates], 0)
            label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # トレーニングの例からグローバルリストに各要素を追加します。.
            targets.append(target_word)
            contexts.append(context)
            labels.append(label)
    return targets, contexts, labels

In [16]:
# コーパスのDL
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [17]:
# コーパスの文章
with open(path_to_file) as f: 
    lines = f.read().splitlines()
for line in lines[:20]:
    print(line)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.


In [18]:
# ベクトル化のためにDatasetへ変換
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))
text_ds

<FilterDataset shapes: (), types: tf.string>

In [19]:
# コーパスから文をベクトル化する
# カスタム標準化機能を作成して、テキストを下げます。 
# 句読点の削除
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')

# サイズと単語数を順番に定義します。.
vocab_size = 4096
sequence_length = 10

# テキストベクトル化レイヤーを使用して、文字列を正規化、分割、およびマップします。
#整数、output_sequence_length lengthを設定して、すべてのサンプルを同じ長さにパッドします。.
vectorize_layer = TextVectorization(
    standardize=custom_standardization, # 正規化
    max_tokens=vocab_size, # 語彙の最大値
    output_mode='int', 
    output_sequence_length=sequence_length) # 出力する文章の長さ

In [20]:
# テキストデータセットでadaptを呼び出して、語彙を作成します。
vectorize_layer.adapt(text_ds.batch(1024))

In [21]:
# 語彙にアクセス
# 参照用に作成したボキャブラリーを保存します。.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your']


In [22]:
# ベクトル生成
def vectorize_text(text):
    text = tf.expand_dims(text, -1)
    return tf.squeeze(vectorize_layer(text))

# text_dsでデータを修正します。.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [23]:
# シーケンスの所得
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))
sequences

32777


[array([ 89, 270,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([138,  36, 982, 144, 673, 125,  16, 106,   0,   0]),
 array([34,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 array([106, 106,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([ 89, 270,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([   7,   41,   34, 1286,  344,    4,  200,   64,    4, 3690]),
 array([34,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 array([1286, 1286,    0,    0,    0,    0,    0,    0,    0,    0]),
 array([ 89, 270,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([  89,    7,   93, 1187,  225,   12, 2442,  592,    4,    2]),
 array([34,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 array([  36, 2655,   36, 2655,    0,    0,    0,    0,    0,    0]),
 array([ 89, 270,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([  72,   79,  506,   27,    3,   56,   24, 1390,   57,   40]),
 array([644,   9,   1,   0,   0,   0,   0,   0,   0,   0]),
 array([34,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 array([  32,   54, 2863,  885

In [24]:
# 先端の確認
for seq in sequences[:5]:
    print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
[138  36 982 144 673 125  16 106   0   0] => ['before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', '', '']
[34  0  0  0  0  0  0  0  0  0] => ['all', '', '', '', '', '', '', '', '', '']
[106 106   0   0   0   0   0   0   0   0] => ['speak', 'speak', '', '', '', '', '', '', '', '']
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']


In [25]:
# 例
targets, contexts, labels = generate_training_data(
    sequences=sequences, 
    window_size=2, 
    num_ns=4, 
    vocab_size=vocab_size, 
    seed=SEED)
print(len(targets), len(contexts), len(labels))

100%|██████████| 32777/32777 [00:08<00:00, 3915.31it/s]

65324 65324 65324


In [26]:
# バッチ処理
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: (((1024,), (1024, 5, 1)), (1024, 5)), types: ((tf.int32, tf.int64), tf.int64)>


In [27]:
# パフォーマンス向上
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset shapes: (((1024,), (1024, 5, 1)), (1024, 5)), types: ((tf.int32, tf.int64), tf.int64)>


In [28]:
# クラス化
class Word2Vec(Model):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2Vec, self).__init__()
        self.target_embedding = Embedding(vocab_size, 
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding", )
        self.context_embedding = Embedding(vocab_size, 
                                       embedding_dim, 
                                       input_length=num_ns+1)
        self.dots = Dot(axes=(3,2))
        self.flatten = Flatten()

    def call(self, pair):
        target, context = pair
        we = self.target_embedding(target)
        ce = self.context_embedding(context)
        dots = self.dots([ce, we])
        return self.flatten(dots)

In [29]:
# 損失関数の定義
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

In [30]:
# 次元を１２８で走らせる
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [31]:
# テンソルボード
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [32]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
63/63 [==============================] - 1s 7ms/step - loss: 1.6089 - accuracy: 0.2188
Epoch 2/20
63/63 [==============================] - 0s 4ms/step - loss: 1.5922 - accuracy: 0.5963
Epoch 3/20
63/63 [==============================] - 0s 4ms/step - loss: 1.5528 - accuracy: 0.6431
Epoch 4/20
63/63 [==============================] - 0s 5ms/step - loss: 1.4752 - accuracy: 0.5937
Epoch 5/20
63/63 [==============================] - 0s 5ms/step - loss: 1.3770 - accuracy: 0.5852
Epoch 6/20
63/63 [==============================] - 0s 4ms/step - loss: 1.2782 - accuracy: 0.6064
Epoch 7/20
63/63 [==============================] - 0s 5ms/step - loss: 1.1858 - accuracy: 0.6379
Epoch 8/20
63/63 [==============================] - 0s 5ms/step - loss: 1.1005 - accuracy: 0.6725
Epoch 9/20
63/63 [==============================] - 0s 5ms/step - loss: 1.0215 - accuracy: 0.7064
Epoch 10/20
63/63 [==============================] - 0s 5ms/step - loss: 0.9483 - accuracy: 0.7365
Epoch 11/20
63/63 [

In [33]:
%tensorboard --logdir logs

UsageError: Line magic function `%tensorboard` not found.


In [34]:
# 重みの所得と語彙の提供
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()
weights

array([[-0.02537469,  0.00112855, -0.04549183, ...,  0.04601158,
         0.04681564, -0.03546917],
       [ 0.12380849,  0.09057274,  0.21093182, ..., -0.09339179,
         0.04083184,  0.0188039 ],
       [-0.35800824,  0.18911447, -0.09829159, ..., -0.02338062,
        -0.44768295,  0.07628717],
       ...,
       [ 0.14208461, -0.21986766,  0.00160606, ...,  0.35863736,
        -0.17012271, -0.06212271],
       [ 0.05854943, -0.19218849,  0.05547474, ...,  0.00108582,
         0.15975246,  0.08667073],
       [-0.10363265, -0.32955852,  0.24333785, ..., -0.05076605,
         0.05577997,  0.33788833]], dtype=float32)

In [35]:
# 'vectors.tsv', 'metadata.tsv'をDLして分析
try:
    from google.colab import files
    files.download('vectors.tsv')
    files.download('metadata.tsv')
except Exception as e:
    pass

# 【問題3】Iris（2値分類）をKerasで学習
TensorFlowによるIrisデータセットに対する2値分類をKerasに書き換えてください。

In [36]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


iris = load_iris()
X = iris.data[:100]
y = iris.target

y_reduce = y[:100]
y_reduce=  y_reduce.reshape(-1,1) == np.arange(2)

In [37]:
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X,y_reduce, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [38]:
# モデル生成
def generate_model(X):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=X_train.shape[1:]))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(2, activation="softmax"))
    return model

model = generate_model(X_train)


# コンパイル
def generate_compile(model):
    model.compile(loss="binary_crossentropy",
                 optimizer="RMSprop", 
                 metrics = ['accuracy'])
    return model
    
model = generate_compile(model)

# 学習とpredto評価を丸める
def learning_evaluate(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train,
         batch_size=10,
         epochs=20,
         validation_data=(X_test, y_test),
         verbose=1)
    pred = model.predict(X_test)
    scores = model.evaluate(X_test, y_test)
    return pred, scores

pred, scores = learning_evaluate(model, X_train, y_train, X_val, y_val)

Epoch 1/20
7/7 [==============================] - 1s 100ms/step - loss: 0.6862 - accuracy: 0.5640 - val_loss: 0.5341 - val_accuracy: 1.0000
Epoch 2/20
7/7 [==============================] - 0s 8ms/step - loss: 0.4785 - accuracy: 1.0000 - val_loss: 0.3809 - val_accuracy: 1.0000
Epoch 3/20
7/7 [==============================] - 0s 7ms/step - loss: 0.2835 - accuracy: 1.0000 - val_loss: 0.1652 - val_accuracy: 1.0000
Epoch 4/20
7/7 [==============================] - 0s 7ms/step - loss: 0.1706 - accuracy: 1.0000 - val_loss: 0.0744 - val_accuracy: 1.0000
Epoch 5/20
7/7 [==============================] - 0s 8ms/step - loss: 0.0709 - accuracy: 1.0000 - val_loss: 0.0346 - val_accuracy: 1.0000
Epoch 6/20
7/7 [==============================] - 0s 7ms/step - loss: 0.0345 - accuracy: 1.0000 - val_loss: 0.0156 - val_accuracy: 1.0000
Epoch 7/20
7/7 [==============================] - 0s 7ms/step - loss: 0.0183 - accuracy: 1.0000 - val_loss: 0.0075 - val_accuracy: 1.0000
Epoch 8/20
7/7 [================

In [39]:
print(pred)
print("-"*50)
print(scores[0]) # loss
print(scores[1]) # accuracy

[[1.0000000e+00 3.8353470e-10]
 [1.0000000e+00 7.0785161e-10]
 [2.0170334e-11 1.0000000e+00]
 [5.0730709e-09 1.0000000e+00]
 [1.8430745e-11 1.0000000e+00]
 [1.0000000e+00 1.5768058e-10]
 [1.9217615e-11 1.0000000e+00]
 [2.1789551e-10 1.0000000e+00]
 [1.0000000e+00 1.5260954e-11]
 [5.3142135e-12 1.0000000e+00]
 [1.0000000e+00 1.8429031e-10]
 [1.0000000e+00 4.1939779e-10]
 [1.0000000e+00 1.2340061e-08]
 [1.0000000e+00 6.2051347e-10]
 [1.0000000e+00 1.3628565e-09]
 [1.0000000e+00 5.8206745e-10]]
--------------------------------------------------
2.64738300757017e-05
1.0


# 【問題4】Iris（多値分類）をKerasで学習
TensorFlowによるIrisデータセットに対する3値分類をKerasに書き換えてください。

In [40]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


iris = load_iris()
X = iris.data
y = iris.target

y_hot = to_categorical(y)

In [41]:
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X,y_hot, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train2, X_val2, y_train2, y_val2 = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
X_train2.shape, y_train2.shape, X_val2.shape

((96, 4), (96, 3), (24, 4))

In [42]:
# 情報を１度リセット
tf.keras.backend.clear_session()

In [43]:
# モデル生成
def generate_model2(X):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=X.shape[1:]))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))
    return model

model2 = generate_model2(X_train2)
    
# コンパイル
def compile_model2(model):
    model.compile(loss='categorical_crossentropy',
                 optimizer='RMSprop',
                 metrics=['accuracy'])
    return model

model2 = compile_model2(model2)

# 学習〜予測〜評価
def learning_evaluate2(model, X_train, y_train, X_val, y_val):
    model.fit(X_train, y_train, 
             batch_size=10,
             epochs=100,
             verbose=2,
             validation_data=(X_val, y_val))
    pred = model.predict(X_val)
    score = model.evaluate(X_val, y_val)
    return pred, score

pred2, score2 = learning_evaluate2(model2, X_train2, y_train2, X_val2, y_val2)

Epoch 1/100
10/10 - 1s - loss: 1.0125 - accuracy: 0.5625 - val_loss: 0.8668 - val_accuracy: 0.7083
Epoch 2/100
10/10 - 0s - loss: 0.7759 - accuracy: 0.7188 - val_loss: 0.5987 - val_accuracy: 0.7917
Epoch 3/100
10/10 - 0s - loss: 0.5762 - accuracy: 0.7604 - val_loss: 0.4737 - val_accuracy: 0.9167
Epoch 4/100
10/10 - 0s - loss: 0.4424 - accuracy: 0.8125 - val_loss: 0.3895 - val_accuracy: 0.8750
Epoch 5/100
10/10 - 0s - loss: 0.4087 - accuracy: 0.8229 - val_loss: 0.3459 - val_accuracy: 0.9167
Epoch 6/100
10/10 - 0s - loss: 0.3493 - accuracy: 0.8750 - val_loss: 0.3201 - val_accuracy: 0.8750
Epoch 7/100
10/10 - 0s - loss: 0.3533 - accuracy: 0.8646 - val_loss: 0.2873 - val_accuracy: 0.9167
Epoch 8/100
10/10 - 0s - loss: 0.2960 - accuracy: 0.8854 - val_loss: 0.4654 - val_accuracy: 0.7083
Epoch 9/100
10/10 - 0s - loss: 0.2208 - accuracy: 0.9479 - val_loss: 0.2399 - val_accuracy: 0.9167
Epoch 10/100
10/10 - 0s - loss: 0.2460 - accuracy: 0.9271 - val_loss: 0.2657 - val_accuracy: 0.9167
Epoch 11/

10/10 - 0s - loss: 0.0760 - accuracy: 0.9792 - val_loss: 0.2982 - val_accuracy: 0.8333
Epoch 84/100
10/10 - 0s - loss: 0.0702 - accuracy: 0.9792 - val_loss: 0.2559 - val_accuracy: 0.9167
Epoch 85/100
10/10 - 0s - loss: 0.0756 - accuracy: 0.9688 - val_loss: 0.2889 - val_accuracy: 0.8333
Epoch 86/100
10/10 - 0s - loss: 0.0948 - accuracy: 0.9479 - val_loss: 0.3522 - val_accuracy: 0.9167
Epoch 87/100
10/10 - 0s - loss: 0.0743 - accuracy: 0.9688 - val_loss: 0.3272 - val_accuracy: 0.9167
Epoch 88/100
10/10 - 0s - loss: 0.1418 - accuracy: 0.9479 - val_loss: 0.2526 - val_accuracy: 0.9167
Epoch 89/100
10/10 - 0s - loss: 0.0615 - accuracy: 0.9792 - val_loss: 0.5146 - val_accuracy: 0.9167
Epoch 90/100
10/10 - 0s - loss: 0.0420 - accuracy: 0.9688 - val_loss: 0.3198 - val_accuracy: 0.9167
Epoch 91/100
10/10 - 0s - loss: 0.0259 - accuracy: 0.9896 - val_loss: 0.3826 - val_accuracy: 0.9167
Epoch 92/100
10/10 - 0s - loss: 0.1206 - accuracy: 0.9688 - val_loss: 0.3411 - val_accuracy: 0.9167
Epoch 93/100


In [44]:
print(pred2)
print("-"*50)
print(score2[0]) # loss
print(score2[1]) # accuracy

[[1.00000000e+00 5.49315295e-08 1.64637401e-11]
 [6.40882661e-07 2.68659950e-03 9.97312725e-01]
 [1.00000000e+00 2.03801207e-08 3.72225861e-12]
 [1.19039854e-04 9.97758985e-01 2.12195609e-03]
 [1.15952971e-04 9.98022676e-01 1.86132535e-03]
 [2.02204130e-04 9.94720459e-01 5.07734763e-03]
 [1.59128231e-05 1.13239679e-02 9.88660097e-01]
 [1.71338172e-06 4.06134967e-03 9.95936990e-01]
 [9.99999881e-01 1.61814199e-07 6.59364646e-11]
 [6.37918129e-04 1.20606482e-01 8.78755629e-01]
 [4.23142657e-04 9.91836727e-01 7.74006080e-03]
 [7.95990118e-08 9.91526176e-04 9.99008477e-01]
 [1.00000000e+00 9.61725188e-09 1.28401261e-12]
 [7.12140718e-07 2.67367531e-03 9.97325540e-01]
 [9.99999881e-01 7.50362474e-08 2.46567558e-11]
 [3.27313252e-08 6.78292883e-04 9.99321699e-01]
 [1.73879729e-04 9.96215999e-01 3.61016998e-03]
 [1.00000000e+00 3.10752313e-10 1.14554447e-14]
 [1.33800576e-03 2.37527803e-01 7.61134148e-01]
 [9.99999881e-01 7.99134625e-08 2.24187786e-11]
 [9.99999881e-01 7.41344692e-08 2.012494

# 【問題5】House PricesをKerasで学習
TensorFlowによるHouse Pricesデータセットに対する回帰をKerasに書き換えてください。

In [45]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

dir_path = '/Users/yuki.tatsuoka/Downloads/house-prices-advanced-regression-techniques (1)/'
house_price_train = pd.read_csv(dir_path + 'train.csv')
house_price_test = pd.read_csv(dir_path + 'test.csv')
pd.set_option('display.max_rows', 300)


y = house_price_train['SalePrice']
X = house_price_train.drop(['SalePrice'], axis=1)

# object型の変換と欠損値を埋めることが必須条件
X = X.fillna(0)
X = pd.get_dummies(X)

In [46]:
X.isnull().sum().sum()

0

In [47]:
# 対数と標準化
y_log = np.log(y)

In [48]:
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X,y_log, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train3, X_val3, y_train3, y_val3 = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
X_train3.shape, y_train3.shape, X_val3.shape

((934, 305), (934,), (234, 305))

In [49]:
# 情報を１度リセット
tf.keras.backend.clear_session()

# モデル生成
def generate_model3(X):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=X.shape[1:]))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    return model

model3 = generate_model3(X_train3)
    
# コンパイル
def compile_model3(model):
    model.compile(loss='mean_squared_error',
                 optimizer='RMSprop')
    return model

model3 = compile_model3(model3)

# 学習〜予測〜評価
def learning_evaluate3(model, X_train, y_train, X_val, y_val):
    model.fit(X_train, y_train, 
             batch_size=10,
             epochs=50,
             verbose=2,
             validation_data=(X_val, y_val))
    pred = model.predict(X_val)
    score = model.evaluate(X_val, y_val)
    return pred, score

pred3, score3 = learning_evaluate3(model3, X_train3, y_train3, X_val3, y_val3)

Epoch 1/50
94/94 - 1s - loss: 624.4451 - val_loss: 92.9573
Epoch 2/50
94/94 - 0s - loss: 204.5008 - val_loss: 14.8860
Epoch 3/50
94/94 - 0s - loss: 51.7791 - val_loss: 64.1671
Epoch 4/50
94/94 - 0s - loss: 187.0215 - val_loss: 14.8193
Epoch 5/50
94/94 - 0s - loss: 20.2332 - val_loss: 6.9963
Epoch 6/50
94/94 - 0s - loss: 25.3025 - val_loss: 6.9996
Epoch 7/50
94/94 - 0s - loss: 15.5955 - val_loss: 0.8328
Epoch 8/50
94/94 - 0s - loss: 46.4206 - val_loss: 1.6967
Epoch 9/50
94/94 - 0s - loss: 11.8335 - val_loss: 1.9538
Epoch 10/50
94/94 - 0s - loss: 34.3742 - val_loss: 0.4189
Epoch 11/50
94/94 - 0s - loss: 7.2767 - val_loss: 1.6662
Epoch 12/50
94/94 - 0s - loss: 10.0720 - val_loss: 2.2108
Epoch 13/50
94/94 - 0s - loss: 8.7504 - val_loss: 0.3664
Epoch 14/50
94/94 - 0s - loss: 7.1021 - val_loss: 0.3934
Epoch 15/50
94/94 - 0s - loss: 7.3209 - val_loss: 11.0558
Epoch 16/50
94/94 - 0s - loss: 21.1219 - val_loss: 0.5025
Epoch 17/50
94/94 - 0s - loss: 6.6737 - val_loss: 0.2430
Epoch 18/50
94/94 - 

In [50]:
print(pred3)
print("-"*50)
print(score3) # loss

[[13.816034 ]
 [13.366222 ]
 [15.013245 ]
 [14.253233 ]
 [14.018711 ]
 [13.987976 ]
 [13.717012 ]
 [14.651091 ]
 [14.358923 ]
 [13.84026  ]
 [14.377975 ]
 [14.362335 ]
 [12.75123  ]
 [14.0571   ]
 [14.095514 ]
 [13.933168 ]
 [12.547417 ]
 [13.902378 ]
 [13.147242 ]
 [13.229284 ]
 [13.541864 ]
 [13.850788 ]
 [13.779642 ]
 [13.438946 ]
 [13.421415 ]
 [14.135551 ]
 [13.586111 ]
 [14.034977 ]
 [13.876345 ]
 [13.924873 ]
 [13.59984  ]
 [14.369614 ]
 [14.083162 ]
 [13.596947 ]
 [14.227592 ]
 [13.719368 ]
 [13.817446 ]
 [13.9466915]
 [13.534664 ]
 [13.919802 ]
 [13.630209 ]
 [13.766066 ]
 [13.528286 ]
 [14.060547 ]
 [14.533314 ]
 [14.508183 ]
 [13.785784 ]
 [14.1009445]
 [13.522036 ]
 [13.798182 ]
 [13.889698 ]
 [13.582575 ]
 [14.17104  ]
 [14.614214 ]
 [13.655239 ]
 [14.48497  ]
 [14.778734 ]
 [13.6823635]
 [13.553944 ]
 [14.085253 ]
 [15.208967 ]
 [13.930077 ]
 [13.537394 ]
 [14.720989 ]
 [13.782322 ]
 [13.54743  ]
 [13.763994 ]
 [13.761505 ]
 [14.23591  ]
 [13.356344 ]
 [13.423315 ]
 [13.2

# 【問題6】MNISTをKerasで学習
TensorFlowによるMNISTデータセットによる画像の多値分類をKerasに書き換えてください。

In [51]:
from tensorflow.keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape

(60000, 28, 28)

In [52]:
# さらにtrainとvalに分割
X_train4, X_val4, y_train4, y_val4 = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
X_train4.shape, y_train4.shape, X_val4.shape

# ワンホット
y_train4 = to_categorical(y_train4)
y_val4 = to_categorical(y_val4)

# チャネルの追加
X_train4 = X_train4.reshape(-1,28,28,1)
X_val4 = X_val4.reshape(-1,28,28,1)

X_train4.shape, y_train4.shape, X_val4.shape

((48000, 28, 28, 1), (48000, 10), (12000, 28, 28, 1))

In [ ]:
# モデル生成
def generate_model4(X):
    model = Sequential()
    Conv2D, MaxPool2D
    
    model.add(Conv2D(32, kernel_size=(3,3), padding='same', input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
    model.add(Conv2D(64, kernel_size=(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))
    return model

model4 = generate_model4(X_train4)
    
# コンパイル
def compile_model4(model):
    model.compile(loss='categorical_crossentropy',
                 optimizer='RMSprop',
                 metrics=['accuracy'])
    return model

model4 = compile_model4(model4)

# 学習〜予測〜評価
def learning_evaluate4(model, X_train, y_train, X_val, y_val):
    history = model.fit(X_train, y_train, 
             batch_size=200,
             epochs=10,
             verbose=2,
             validation_data=(X_val, y_val))
    pred = model.predict(X_val)
    score = model.evaluate(X_val, y_val)
    return history, pred, score

history, pred4, score4 = learning_evaluate4(model4, X_train4, y_train4, X_val4, y_val4)

Epoch 1/10
240/240 - 18s - loss: 0.7780 - accuracy: 0.8138 - val_loss: 0.0735 - val_accuracy: 0.9807
Epoch 2/10
240/240 - 18s - loss: 0.1231 - accuracy: 0.9686 - val_loss: 0.0714 - val_accuracy: 0.9819
Epoch 3/10
240/240 - 16s - loss: 0.0857 - accuracy: 0.9784 - val_loss: 0.0686 - val_accuracy: 0.9813
Epoch 4/10
240/240 - 16s - loss: 0.0691 - accuracy: 0.9829 - val_loss: 0.0790 - val_accuracy: 0.9814
Epoch 5/10
240/240 - 16s - loss: 0.0548 - accuracy: 0.9862 - val_loss: 0.0573 - val_accuracy: 0.9867
Epoch 6/10
240/240 - 16s - loss: 0.0504 - accuracy: 0.9878 - val_loss: 0.0602 - val_accuracy: 0.9875
Epoch 7/10
240/240 - 16s - loss: 0.0430 - accuracy: 0.9894 - val_loss: 0.0513 - val_accuracy: 0.9877
Epoch 8/10


In [ ]:
model4.summary()

In [ ]:
# 予測数値と損失と評価の確認
print(pred4)
print("-"*50)
print(score4[0]) # loss
print(score4[1]) # accuracy

In [ ]:
# 重みの所得
model4.get_weights()

In [ ]:
import matplotlib.pyplot as plt

# accracy可視化
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

#loss可視化
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

#　valが最初から高いので過学習気味か？？

# 【問題7】（アドバンス課題）PyTorchへの書き換え
4種類の問題をPyTorchに書き換えてください。